In [7]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import VarianceThreshold, RFE, SelectFromModel
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd
melbourne_data = pd.read_csv('melbourne_housing_raw.csv')
print(melbourne_data)
missing_percentage = melbourne_data.isnull().mean() * 100
columns_to_remove = [col for col in missing_percentage.index if missing_percentage[col] > 20 and col != 'Price']
reduced_data = melbourne_data.drop(columns=columns_to_remove)
reduced_data = reduced_data.dropna(subset=['Price'])
X = reduced_data.drop(columns=['Price', 'Date', 'Suburb', 'Type', 'Method', 'SellerG', 'CouncilArea', 'Regionname'])
y = reduced_data['Price']
X.fillna(X.mean(), inplace=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
corr_matrix = X_train.corr().abs()
high_corr_pairs = np.where(corr_matrix > 0.85)
high_corr_features = set([X_train.columns[i] for i in high_corr_pairs[0] if i != high_corr_pairs[1][i]])
X_train_corr_filtered = X_train.drop(columns=high_corr_features)
X_test_corr_filtered = X_test.drop(columns=high_corr_features)
variance_filter = VarianceThreshold(threshold=0.01)
X_train_low_var = variance_filter.fit_transform(X_train_corr_filtered)
X_test_low_var = variance_filter.transform(X_test_corr_filtered)
linear_model = LinearRegression()
rfe_selector = RFE(estimator=linear_model, n_features_to_select=5, step=1)
rfe_selector.fit(X_train_low_var, y_train)
X_train_forward = rfe_selector.transform(X_train_low_var)
X_test_forward = rfe_selector.transform(X_test_low_var)
random_forest_model = RandomForestRegressor(random_state=42)
rfe_backward = RFE(estimator=random_forest_model, n_features_to_select=5, step=1)
rfe_backward.fit(X_train_low_var, y_train)
X_train_backward = rfe_backward.transform(X_train_low_var)
X_test_backward = rfe_backward.transform(X_test_low_var)
rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train, y_train)
important_features = SelectFromModel(rf_model, threshold="mean", prefit=True)
X_train_rf_selected = important_features.transform(X_train)
X_test_rf_selected = important_features.transform(X_test)
def evaluate_model(X_train, X_test, y_train, y_test):
    model = RandomForestRegressor(random_state=42)
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    mse = mean_squared_error(y_test, predictions)
    return mse
print("Accuracy:", evaluate_model(X_train, X_test, y_train, y_test))
print("After High Correlation Filter:", evaluate_model(X_train_corr_filtered, X_test_corr_filtered, y_train, y_test))
print("After Low Variance Filter:", evaluate_model(X_train_low_var, X_test_low_var, y_train, y_test))
print("After Forward Selection:", evaluate_model(X_train_forward, X_test_forward, y_train, y_test))
print("After Backward Elimination:", evaluate_model(X_train_backward, X_test_backward, y_train, y_test))
print("After Random Forest Selection:", evaluate_model(X_train_rf_selected, X_test_rf_selected, y_train, y_test))


           Suburb  Rooms Type      Price Method        SellerG     Date  \
0      Abbotsford      2    h        NaN     SS         Jellis   3/9/16   
1      Abbotsford      2    h  1480000.0      S         Biggin  3/12/16   
2      Abbotsford      2    h  1035000.0      S         Biggin   4/2/16   
3      Abbotsford      3    u        NaN     VB         Rounds   4/2/16   
4      Abbotsford      3    h  1465000.0     SP         Biggin   4/3/17   
...           ...    ...  ...        ...    ...            ...      ...   
34852  Yarraville      4    h  1480000.0     PI            Jas  24/2/18   
34853  Yarraville      2    h   888000.0     SP        Sweeney  24/2/18   
34854  Yarraville      2    t   705000.0      S            Jas  24/2/18   
34855  Yarraville      3    h  1140000.0     SP  hockingstuart  24/2/18   
34856  Yarraville      2    h  1020000.0     PI             RW  24/2/18   

       Distance  Postcode  Bedroom2  Bathroom  Car  Landsize  BuildingArea  \
0           2.5    30

/usr/local/lib/python3.10/dist-packages/sklearn/feature_selection/_rfe.py:291: UserWarning: Found n_features_to_select=5 > n_features=4. There will be no feature selection and all features will be kept.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_selection/_rfe.py:291: UserWarning: Found n_features_to_select=5 > n_features=4. There will be no feature selection and all features will be kept.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


Accuracy: 143875373039.6255
After High Correlation Filter: 143875373039.6255
After Low Variance Filter: 143875373039.6255
After Forward Selection: 143875373039.6255
After Backward Elimination: 143875373039.6255
After Random Forest Selection: 147811757361.13766
